In [150]:
import pandas as pd
from datetime import datetime, date

In [151]:
def make_df(df, year=[2021],bins=[20,150],
            x='Category', quantile=0.1, topcut=20,
            aggregate_by='average'):
    # load/prepare datafile
    
    df = df.loc[df['Date'].dt.year.isin(year)]
    df = df.set_index('Date')

    # choose x, bins
    if bins:
        if bins == 'Time Comparison':
            grabs = df.resample('M').count()
            most_recent = grabs.iloc[-int(grabs.shape[0]/2):].index
            previous = grabs.loc[~grabs.index.isin(most_recent)].index
            labels = [f'before {previous[-1].date()}', f'after {previous[-1].date()}']
            
            if x != 'Date':
                dff = df.groupby(x).resample('M')[['Amount']].mean()
            else:
                dff = df.resample('M')[['Amount']].sum()
            dff = dff.reset_index()
            dff.loc[dff.Date.isin(most_recent), 'Bin'] = labels[1]
            dff.loc[dff.Date.isin(previous), 'Bin'] = labels[0]
            
        else:
            labels = [f'less than {bins[0]}', f'between {bins[0]} and {bins[1]}', f'greater than {bins[1]}']
            if x != 'Date':
                dff = df.loc[(df.Amount < bins[0])].groupby(x).resample('M')[['Amount']].sum()
                dff.columns = [labels[0]]
                dff[labels[1]] = df.loc[(df.Amount >= bins[0]) &
                                        (df.Amount < bins[1])].groupby(x).resample('M')[['Amount']].sum()
                dff[labels[2]] = df.loc[(df.Amount > bins[1])].groupby(x).resample('M')[['Amount']].sum()
                dff = dff.reset_index()
                dff = dff.melt(id_vars=[x, 'Date'], value_name='Amount', var_name='Bin')
            else:
                dff = df.loc[(df.Amount < bins[0])].resample('M')[['Amount']].sum()
                dff.columns = [labels[0]]
                dff[labels[1]] = df.loc[(df.Amount >= bins[0]) &
                                        (df.Amount < bins[1])].resample('M')[['Amount']].sum()
                dff[labels[2]] = df.loc[(df.Amount > bins[1])].resample('M')[['Amount']].sum()
                dff = dff.reset_index()
                dff = dff.melt(id_vars=['Date'], value_name='Amount', var_name='Bin')
    else:
        
        if x != 'Date':
            dff = df.groupby(x).resample('M')[['Amount']].sum()
            dff = dff.reset_index()
        else:
            dff = df.resample('M')[['Amount']].sum()
            dff = dff.reset_index()
            
    if quantile:
        mymap1 = (dff.resample(on='Date', rule='M').sum()['Amount'] <= dff.resample(on='Date', rule='M').sum().quantile(quantile)[0]).to_dict()
        mymap2 = (dff.resample(on='Date', rule='M').sum()['Amount'] >= dff.resample(on='Date', rule='M').sum().quantile(1-quantile)[0]).to_dict()

        mymap = {}
        for key, val in mymap1.items():
            if val == True:
                mymap[key] = 'Bottom'
            elif mymap2[key] == True:
                mymap[key] = 'Top'
            else:
                mymap[key] = 'Middle'
        dff['Quantile'] = dff['Date'].map(mymap)

    # cut off small - value categories (x)
    if x != 'Date':
        dfftop = dff.groupby(x)[['Amount']].sum().sort_values('Amount', ascending=False)[:topcut]
        dfftop = dfftop.reset_index()
        dff1 = dff.loc[dff[x].isin(dfftop[x])]
        dff2 = dff.loc[~dff[x].isin(dfftop[x])]
        if bins and quantile:
            dffbot = pd.DataFrame(dff2.groupby(['Date', 'Bin', 'Quantile'])['Amount'].sum()).reset_index()
        elif quantile:
            dffbot = pd.DataFrame(dff2.groupby(['Date', 'Quantile'])['Amount'].sum()).reset_index()
        elif bins:
            dffbot = pd.DataFrame(dff2.groupby(['Date', 'Bin'])['Amount'].sum()).reset_index()
        else:
            dffbot = pd.DataFrame(dff2.groupby(['Date'])['Amount'].sum()).reset_index()
        dffbot[x] = 'Other'
        dff = pd.concat([dff1, dffbot])

    # collapse onto groupby
    if aggregate_by == 'average':
        if bins and quantile:
            dff = dff.groupby([x, 'Quantile', 'Bin'])[['Amount']].mean().reset_index()
        elif quantile:
            dff = dff.groupby([x, 'Quantile'])[['Amount']].mean().reset_index()
        elif bins:
            dff = dff.groupby([x, 'Bin'])[['Amount']].mean().reset_index()
        else:
            dff = dff.groupby([x])[['Amount']].mean().reset_index()
    else:
        if bins and quantile:
            dff = dff.groupby([x, 'Quantile', 'Bin'])[['Amount']].sum().reset_index()
        elif quantile:
            dff = dff.groupby([x, 'Quantile'])[['Amount']].sum().reset_index()
        elif bins:
            dff = dff.groupby([x, 'Bin'])[['Amount']].sum().reset_index()
        else:
            dff = dff.groupby([x])[['Amount']].sum().reset_index()

    # sort results, up to 3 sort categories
    if x != 'Date':
        sorter1 = dff.groupby(x)['Amount'].sum().sort_values(ascending=False).index
        sorterIndex1 = dict(zip(sorter1, range(len(sorter1))))
    if quantile:
        sorter2 = ['Top', 'Middle', 'Bottom']
        sorterIndex2 = dict(zip(sorter2, range(len(sorter2))))
    if bins:
        sorter3 = labels
        sorterIndex3 = dict(zip(sorter3, range(len(sorter3))))
        
    if bins and x != 'Date' and quantile:
        sorters = [sorterIndex1, sorterIndex2, sorterIndex3]
    elif bins and quantile:
        sorters = [sorterIndex2, sorterIndex3]
    elif quantile:
        sorters = [sorterIndex2]
    elif bins:
        sorters = [sorterIndex3]
        
    if x != 'Date':
        dff['sort1'] = dff[x].map(sorterIndex1)
    if quantile:
        dff['sort2'] = dff.Quantile.map(sorterIndex2)
    if bins:
        dff['sort3'] = dff.Bin.map(sorterIndex3)
        if x != 'Date' and quantile:
            dff = dff.sort_values(['sort1', 'sort2', 'sort3']).reset_index(drop=True)
            dff = dff.drop(['sort1', 'sort2', 'sort3'], axis=1)
        elif quantile:
            dff = dff.sort_values(['sort2', 'sort3']).reset_index(drop=True)
            dff = dff.drop(['sort2', 'sort3'], axis=1)
        elif x != 'Date':
            dff = dff.sort_values(['sort1', 'sort3']).reset_index(drop=True)
            dff = dff.drop(['sort1', 'sort3'], axis=1)
        else:
            dff = dff.sort_values(['sort3']).reset_index(drop=True)
            dff = dff.drop(['sort3'], axis=1)
    else:
        if x != 'Date' and quantile:
            dff = dff.sort_values(['sort1', 'sort2']).reset_index(drop=True)
            dff = dff.drop(['sort1', 'sort2'], axis=1)
            sorters = [sorterIndex1, sorterIndex2]
            labels = None
        elif quantile:
            dff = dff.sort_values(['sort2']).reset_index(drop=True)
            dff = dff.drop(['sort2'], axis=1)
            sorters = [sorterIndex2]
            labels = None
        elif x != 'Date':
            dff = dff.sort_values(['sort1']).reset_index(drop=True)
            dff = dff.drop(['sort1'], axis=1)
            sorters = [sorterIndex1]
            labels = None
        else:
            sorters = None
            labels = None
    return dff, sorters, labels

In [152]:
df = pd.read_csv('../data/transactions.csv', parse_dates=['Date'])

# df = process_data(df, pivot_dates=False)

categories = list(df.Category.unique())
income = ['Income', 'Paycheck', 'Transfer', 'Federal Tax', 'Taxes', 'Rental Income', 'Interest Income']
taxes = ['Federal Tax', 'Taxes']
internal_acc = ['Credit Card Payment', 'Transfer', 'Financial']
non_expense =  income + taxes + internal_acc
expenses = [i for i in categories if i not in non_expense]

df = df.loc[df['Category'].isin(expenses)]
df = df.loc[df['Amount'] < 8000]

# Primary Chart Data

In [153]:
dff, _, _ = make_df(df, x='Date', year=[2021], quantile=0.1, bins=None)
monthly = dff.set_index("Date")
monthly.columns = ['Decile', 'Amount']


for quantile, label in zip([.25,.5], ['Quartile', 'Median']):
    dff, _, _ = make_df(df, x='Date', year=[2021], quantile=quantile, bins=None)
    dff = dff.set_index("Date")
    monthly[label] = dff['Quantile']

monthly.index = monthly.index.month
monthly = monthly.reset_index()
monthly = monthly.sort_values('Date')

monthly = monthly.reset_index(drop=True)
monthly = monthly[['Date', 'Amount', 'Decile', 'Quartile', 'Median']]
monthly['Amount'] = round(monthly['Amount'])
# monthly.to_json('static_data/monthly_spending.json')

# Secondary Chart Data

In [167]:
dff, _, _ = make_df(df, x='Category', year=[2021], quantile=0.1, bins=None)
dff = round(dff)
dff.to_json('static_data/category_spending_10.json')
df1 = pd.concat({"Top/Bottom 10%": dff}, axis=1, names=["Quantile", "Data"])

dff, _, _ = make_df(df, x='Category', year=[2021], quantile=0.25, bins=None)
dff = round(dff)
df2 = pd.concat({"Top/Bottom Quartile": dff}, axis=1, names=["Quantile", "Data"])

dff, _, _ = make_df(df, x='Category', year=[2021], quantile=0.5, bins=None)
dff = round(dff)
df3 = pd.concat({"Above/Bellow Median": dff}, axis=1, names=["Quantile", "Data"])

dff, _, _ = make_df(df, x='Category', year=[2021], quantile=None, bins=None)
dff = round(dff)
df4 = pd.concat({"None": dff}, axis=1, names=["Quantile", "Data"])

In [182]:
dff

,Category,Amount
0,Hotel,1565.0
1,Student Loan,1091.0
2,Home Services,596.0
3,Air Travel,544.0
4,Mortgage & Rent,539.0
5,Groceries,506.0
6,Other,416.0
7,Restaurants,265.0
8,Sporting Goods,259.0
9,Shopping,235.0


In [175]:
category.to_json('static_data/category_spending.json', orient='records')

In [176]:
category.to_json(orient='records')

'[{"(\'Above\\/Bellow Median\', \'Amount\')":null,"(\'Above\\/Bellow Median\', \'Category\')":null,"(\'Above\\/Bellow Median\', \'Quantile\')":null,"(\'None\', \'Amount\')":null,"(\'None\', \'Category\')":null,"(\'Top\\/Bottom 10%\', \'Amount\')":2878.0,"(\'Top\\/Bottom 10%\', \'Category\')":"Hotel","(\'Top\\/Bottom 10%\', \'Quantile\')":"Top","(\'Top\\/Bottom Quartile\', \'Amount\')":null,"(\'Top\\/Bottom Quartile\', \'Category\')":null,"(\'Top\\/Bottom Quartile\', \'Quantile\')":null},{"(\'Above\\/Bellow Median\', \'Amount\')":null,"(\'Above\\/Bellow Median\', \'Category\')":null,"(\'Above\\/Bellow Median\', \'Quantile\')":null,"(\'None\', \'Amount\')":null,"(\'None\', \'Category\')":null,"(\'Top\\/Bottom 10%\', \'Amount\')":1128.0,"(\'Top\\/Bottom 10%\', \'Category\')":"Hotel","(\'Top\\/Bottom 10%\', \'Quantile\')":"Middle","(\'Top\\/Bottom Quartile\', \'Amount\')":null,"(\'Top\\/Bottom Quartile\', \'Category\')":null,"(\'Top\\/Bottom Quartile\', \'Quantile\')":null},{"(\'Above\\/Be